In [6]:
#  resgister at https://icd.who.int/icdapi to get your client_id and client_secret
token_endpoint = 'https://icdaccessmanagement.who.int/connect/token'
client_id = ''
client_secret = ''
scope = 'icdapi_access'
grant_type = 'client_credentials'

In [7]:
import requests
import json

# get the OAUTH2 token
payload = {'client_id': client_id, 'client_secret': client_secret, 'scope': scope, 'grant_type': grant_type}

# request for access token
r = requests.post(token_endpoint, data=payload, verify=False).json()
token = r['access_token']

# set HTTP header fields
headers = {'Authorization':  'Bearer '+ token, 
        'Accept': 'application/json', 
        'Accept-Language': 'en',
        'API-Version': 'v2'}

requests.packages.urllib3.disable_warnings()

# # access ICD API
# # uri = 'https://id.who.int/icd/release/11/2023-01/mms/142052508' # URI for pneumonia ICD11 CA40
# # uri='http://id.who.int/icd/release/11/2023-01/mms/1323682030' # URI for pneumonia ICD11 CA40.0
# # uri = "http://id.who.int/icd/release/11/2023-01/mms/1258650683" # URI for pneumonia ICD11 CA40.00
# # uri = 'https://id.who.int/icd/release/11/2023-01/mms/codeinfo/XN8LS'
# # uri = 'http://id.who.int/icd/release/11/2023-01/mms/1674014893' # XB25
# # uri = 'http://id.who.int/icd/release/11/2023-01/mms/622600769'
# # ICD API endpoint
# uri = 'https://id.who.int/icd/release/11/2020-09/mms/search?q=XN8LS'
# make request           
# r = requests.get(uri, headers=headers, verify=False)
# # print the result in pretty format
# print (json.dumps(r.json(), indent=4))	

Some examples of the response from icd-api


In [8]:
uri = 'http://id.who.int/icd/release/11/2023-01/mms/codeinfo/1A03.1/MG50.26'
r = requests.get(uri, headers=headers, verify=False)
# print (json.dumps(r.json(),indent=4))
json_response = r.json()
json_response.keys()
# json_response['code'], json_response['stemId']

dict_keys(['@context', '@id', 'code', 'stemId', 'stemCode', 'associatedWith'])

In [9]:
uri = 'http://id.who.int/icd/release/11/2023-01/mms/codeinfo/KB24&XN0KC'
r = requests.get(uri, headers=headers, verify=False)
first_response = r.json()
print(first_response)
print (json.dumps(r.json(),indent=4))

{'@context': 'http://id.who.int/icd/contexts/contextForCodeInfo.json', '@id': 'http://id.who.int/icd/release/11/2023-01/mms/codeinfo/KB24%26XN0KC', 'code': 'KB24&XN0KC', 'stemId': 'http://id.who.int/icd/release/11/2023-01/mms/594985340', 'stemCode': 'KB24', 'infectiousAgent': ['XN0KC']}
{
    "@context": "http://id.who.int/icd/contexts/contextForCodeInfo.json",
    "@id": "http://id.who.int/icd/release/11/2023-01/mms/codeinfo/KB24%26XN0KC",
    "code": "KB24&XN0KC",
    "stemId": "http://id.who.int/icd/release/11/2023-01/mms/594985340",
    "stemCode": "KB24",
    "infectiousAgent": [
        "XN0KC"
    ]
}


The code defines three functions that retrieve data from the ICD-11 API:

1. `get_by_codeinfo(codestring, context)`: This function takes a code string and a context as input, and retrieves additional data about the code from the ICD-11 API using the code string. The function returns a JSON object containing the original code string, the context, and any additional data that was retrieved (such as stemId, laterality, hasManifestation, etc.).

2. `get_by_stemid(stemId, context)`: This function takes a stemId and a context as input, and retrieves the code and title associated with the stemId from the ICD-11 API. The function returns a JSON object containing the code, title, and context.

3. `get_icd11_description(response_json, context)`: This function takes a JSON object and a context as input, and retrieves additional data about the code from the JSON object. The function checks if the JSON object contains keys for 'stemId', 'laterality', 'hasManifestation', 'hasCausingCondition', 'associatedWith', or 'infectiousAgent', and if so, it calls the `get_by_codeinfo` and `get_by_stemid` functions to retrieve additional data. The function returns a JSON object containing the original data from the JSON object, as well as any additional data that was retrieved.

4. the final ouptut is a pandas dataframe with stemId and postcoordination context in separate rows.

Note that the code uses the `requests` library to make HTTP requests to the ICD-11 API, and the `json` library to parse JSON data. The code also uses the `headers` variable to specify the headers to be sent with the HTTP requests.

In [10]:
import pandas as pd
def get_by_codeinfo(codestring, context) -> json:
    uri = f'http://id.who.int/icd/release/11/2023-01/mms/codeinfo/{codestring}'
    # print(uri)
    r = requests.get(uri, headers=headers, verify=False)
    if r.status_code == 200:
        response_json = r.json()
        # only return code and title from response_json in json format
        return_json = {'context' :context, 'code_string' : codestring, }
        for key in response_json.keys():
            if key in ['stemId', 'laterality', 'hasManifestation', 'hasCausingCondition', 'associatedWith', 'infectiousAgent']:
                return_json[key] = response_json[key]
        return return_json
    else:
        return None
def get_by_stemid(stemId, context) -> json:
    # takes a stemId and returns the code and title
    uri = f'http://id.who.int/icd/release/11/2023-01/mms/{stemId}'
    r = requests.get(uri, headers=headers, verify=False)

    # check if the response keys have code and title keys
    if r.status_code == 200 and 'code' in r.json().keys() and 'title' in r.json().keys():
        response_json = r.json()
        # only return code and title from response_json in json format
        return_json = { 'context': context, 'code': response_json['code'], 'title': response_json['title']['@value'],  }
        return return_json
    else:
        return None

def get_icd11_description(response_json, context) -> json:
    # returns the code and title for the context from the response_json
    if context in response_json.keys():
        response_json_stemId = {'code_string_orig' : response_json['code_string']} 
        for i in response_json[context]:
            response_json_stemId = { **response_json_stemId,  **get_by_codeinfo(i,context )}
            #  extract stemId from the uri
            stemId = response_json_stemId['stemId'].split('/')[-1]
            response_json_title = { **response_json_stemId,  **get_by_stemid(stemId,context )}
        return response_json_title
    else:
        return None

#  run the code for any icd11 code with postcoordination string
#  from the icd11 browser https://tinyurl.com/5n7vvc3h
#  example: CA40.03&XK9J/1G40
#  example response from api call http://id.who.int/icd/release/11/2023-01/mms/codeinfo/CA40.03%26XK9J%2f1G40:
# {'@context': 'http://id.who.int/icd/contexts/contextForCodeInfo.json',
#  '@id': 'http://id.who.int/icd/release/11/2023-01/mms/codeinfo/CA40.03%26XK9J%2f1G40',
#  'code': 'CA40.03&XK9J/1G40',
#  'stemId': 'http://id.who.int/icd/release/11/2023-01/mms/1250089989',
#  'stemCode': 'CA40.03',
#  'laterality': ['XK9J'],
#  'hasManifestation': ['1G40']}

response_json = None
icd11_post_coord = []

# set the icd11 string to query
# icd11_string = 'CA40.03&XK9J/1G40' #hasManifestation
# icd11_string = '1A03.1/MG50.26' ## associatedWith
icd11_string = 'KB24&XN0KC' ## InfectiousAgent

# get the initial response with stemId and postcoordonation codes
response_json = get_by_codeinfo(icd11_string,'initial' ) 
print(response_json)

if 'stemId' in response_json.keys():
    # print(response_json['stemId'])
    response_json_stemId = {'code_string_orig' : response_json['code_string']}
    #  extract stemId from the uri
    stemId = response_json['stemId'].split('/')[-1]
    response_json_stemId = { **response_json_stemId,  **get_by_stemid(stemId,'stemId' )}
    # add to icd11_final list
    icd11_post_coord.append(response_json_stemId)
    # print(response_json_stemId) 


# get the response for each postcoordination code
for key in response_json.keys():
    if key in ['laterality', 'hasManifestation', 'associatedWith', 'hasCausingCondition', 'infectiousAgent']:
        response_json_title = get_icd11_description(response_json, key)
        if response_json_title is not None:
            icd11_post_coord.append(response_json_title)

# print(icd11_post_coord)
# convert to dataframe
icd11_final_df = pd.DataFrame(icd11_post_coord)
icd11_final_df.head(10)


{'context': 'initial', 'code_string': 'KB24&XN0KC', 'stemId': 'http://id.who.int/icd/release/11/2023-01/mms/594985340', 'infectiousAgent': ['XN0KC']}


,code_string_orig,context,code,title,code_string,stemId
0,KB24&XN0KC,stemId,KB24,Congenital pneumonia,NaN,NaN
1,KB24&XN0KC,infectiousAgent,XN0KC,Streptococcus agalactiae,XN0KC,http://id.who.int/icd/release/11/2023-01/mms/1...
